In [275]:
import os
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

# NV + TA place

In [3]:
nv_place = pd.read_excel(os.path.join("..","..", "data","JS_0601_select_place.xlsx"))

In [4]:
nv_review = pd.read_excel(os.path.join("..","..", "data","JS_0502_reviews_seoul_select.xlsx"))

In [142]:
ta_place = pd.read_excel(os.path.join("..","..", "data","TA_음식점_한글이름_주소_정보.xlsx"), index_col = 0)

In [8]:
ta_review = pd.read_json(os.path.join("..","..", "data","TA_User_Review_Seoul_all_v2.json"))

In [38]:
same_place = pd.read_excel(os.path.join("..","..", "data","JS_0602_same_place_index.xlsx"), index_col = 0)

In [39]:
# 네이버와 트립어드바이저의 같은 place
same_place

,place.id,location_id
2,11605455,2241571
22,10976694,5926853
24,11398664,4032150
25,11663081,3922609
26,11677544,4033804
...,...,...
1032,32817375,4784018
1035,19723511,9209252
1039,33758018,8948116
1063,35162551,9029313


In [37]:
# 트립어드바이저의 location_id를 네이버의 place.id로 대체

In [40]:
same_place.reset_index(inplace = True)

In [45]:
nv_place['place.id'] = nv_place['place.id'].astype(str)
ta_place['location_id'] = ta_place['location_id'].astype(str)
same_place['place.id'] = same_place['place.id'].astype(str)
same_place['location_id'] = same_place['location_id'].astype(str)

In [ ]:
# merge 전 feature 정리

In [49]:
nv_place = nv_place[['place.id', 'place.name','place.category', 'review.count.again', 'review.allcount', 'place.count', 'review.avgcount', 'is_fch', 'rating', 'land.addr', 'lat', 'lng']]

In [52]:
nv_place.head()

,place.id,place.name,place.category,review.count.again,review.allcount,place.count,review.avgcount,is_fch,rating,land.addr,lat,lng
0,11678788,부산아구,"아귀찜,해물찜-일반",20.0,2443,333,7.336336,0,4.150000,서울특별시 강남구 논현동 16-1,37.515770,127.020739
1,11679292,옛날집낙원아구찜,"아귀찜,해물찜-일반",22.0,2443,333,7.336336,0,4.681818,서울특별시 종로구 낙원동 48,37.573505,126.988013
2,11679479,통나무식당 낙원본점,"아귀찜,해물찜-일반",24.0,2443,333,7.336336,0,4.229167,서울특별시 종로구 낙원동 45-3,37.573604,126.988038
3,1188261307,진아구,"아귀찜,해물찜-일반",20.0,2443,333,7.336336,0,4.225000,서울특별시 양천구 신정동 886-6 로데오프라자,37.526858,126.861398
4,12018058,알부자알요리,"아귀찜,해물찜-일반",65.0,2443,333,7.336336,0,4.335821,서울특별시 강남구 역삼동 817-34 재상빌딩,37.500701,127.027850


In [57]:
ta_place = ta_place[['location_id', 'name', 'rating', 'land.addr', 'lat', 'lng']]

In [59]:
ta_place.head()

,location_id,name,rating,land.addr,lat,lng
0,788542,마르코 폴로,4.0,서울특별시 강남구 삼성동 159-1 트레이드타워,37.510404,127.061219
1,4077917,버터핑거 팬케이크 강남점,4.0,서울특별시 서초구 서초동 1317,37.498844,127.025400
2,9732387,빵꼼마,4.5,서울특별시 마포구 동교동 153-32,37.559316,126.924645
3,9379120,명동한식당,5.0,서울특별시 종로구 인사동 92,37.570578,126.985935
4,13833720,스타벅스 리져브 - 용산푸르지오써밋점,4.0,서울특별시 용산구 한강로2가 391,37.527471,126.965248


In [63]:
# 먼저 naver와 merge (네이버-트립어드바이저 겹치는 부분 네이버 정보 사용)
places = same_place.merge(nv_place, on = 'place.id', how = 'inner').drop('index', axis = 1)

In [67]:
# 트립어드바이저 location_id 삭제
places = places.drop('location_id', axis = 1)

In [71]:
# 네이버 플레이스 concat 후 중복제거
places = pd.concat([places, nv_place], axis = 0).drop_duplicates()
places

,place.id,place.name,place.category,review.count.again,review.allcount,place.count,review.avgcount,is_fch,rating,land.addr,lat,lng
0,11605455,가미분식,종합분식,22.0,18625,2121,8.781235,0,4.217391,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,37.558661,126.945960
1,10976694,유래회관,"육류,고기요리-일반",18.0,24041,3347,7.182850,0,4.300000,서울특별시 성동구 홍익동 357 유래회관,37.566438,127.031591
2,11398664,대성갈비,"육류,고기요리-일반",55.0,24041,3347,7.182850,0,4.216667,서울특별시 성동구 성수동1가 668-21,37.547073,127.044028
3,11663081,철길왕갈비살,"육류,고기요리-일반",34.0,24041,3347,7.182850,0,4.400000,서울특별시 마포구 창전동 4-2,37.554546,126.931666
4,11677544,황소고집,"육류,고기요리-일반",49.0,24041,3347,7.182850,0,4.379310,서울특별시 종로구 관철동 11-11,37.568593,126.986046
...,...,...,...,...,...,...,...,...,...,...,...,...
9861,37402391,포시즌스호텔서울 더마켓키친,특급호텔레스토랑,36.0,38,1,38.000000,0,4.276316,서울특별시 종로구 당주동 30,37.570776,126.975422
9862,970385130,빵장수단팥빵 우장산점,빵장수단팥빵,29.0,135,5,27.000000,1,4.545455,서울특별시 강서구 화곡동 1165-1 강서힐스테이트,37.547216,126.836820
9863,37663561,또보겠지떡볶이집 호호시스터점,또보겠지떡볶이집,85.0,344,4,86.000000,1,4.532967,서울특별시 마포구 서교동 396-32,37.548958,126.916815
9865,37914918,장도뚝배기 낙성대점,장도뚝배기,23.0,45,3,15.000000,0,4.340000,서울특별시 관악구 봉천동 1628-18,37.477342,126.960617


In [76]:
ta = ta_place.merge(same_place, on = 'location_id', how = 'left')

In [79]:
# 네이버와 중복되는 부분 제거
ta = ta.loc[ta['place.id'].isnull(), ['location_id', 'name', 'rating', 'land.addr', 'lat', 'lng']]

In [81]:
ta.columns = ['place.id', 'place.name', 'rating', 'land.addr', 'lat', 'lng']
ta

,place.id,place.name,rating,land.addr,lat,lng
0,788542,마르코 폴로,4.0,서울특별시 강남구 삼성동 159-1 트레이드타워,37.510404,127.061219
2,9732387,빵꼼마,4.5,서울특별시 마포구 동교동 153-32,37.559316,126.924645
3,9379120,명동한식당,5.0,서울특별시 종로구 인사동 92,37.570578,126.985935
5,8904596,빠나미,5.0,서울특별시 중구 남대문로5가 541 서울스퀘어,37.555506,126.973768
6,8904578,빠나미,5.0,서울특별시 강남구 삼성동 159-6 한국도심공항터미널,37.509551,127.059469
...,...,...,...,...,...,...
5572,10161356,모모카페 코트야드 서울 남대문,4.5,서울특별시 중구 남대문로4가 17-23 남대문호텔,37.561200,126.976882
5573,3894901,스타벅스 경복궁역점,4.0,서울특별시 종로구 통의동 35-41 우진빌딩,37.577200,126.972778
5574,948337,커피愛,4.5,서울특별시 중구 명동2가 27-1,37.562855,126.985572
5575,9035619,해적선해물부페,4.0,서울특별시 구로구 고척동 76-160 골든타워빌딩,37.497622,126.864067


In [83]:
places = pd.concat([places, ta], axis = 0)

In [95]:
places.reset_index(drop = True, inplace = True)

In [101]:
places.loc[places['place.id'].duplicated(keep = False) == True]

,place.id,place.name,place.category,review.count.again,review.allcount,place.count,review.avgcount,is_fch,rating,land.addr,lat,lng
2861,1129806733,코레우리,"카페,디저트-일반",52.0,76934.0,7641.0,10.068577,0,4.103448,서울특별시 강서구 마곡동 812 서울식물원,37.567789,126.832333
2862,1129806733,코레우리 서울식물원카페,"카페,디저트-일반",52.0,76934.0,7641.0,10.068577,0,4.103448,서울특별시 강서구 마곡동 812 서울식물원,37.567789,126.832333
4159,11736500,허수아비돈까스본점,돈가스-일반,37.0,11397.0,903.0,12.621262,0,4.703704,서울특별시 서초구 서초동 1451-52,37.482072,127.012759
12709,11736500,에머이,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,서울특별시 종로구 관철동 16-9,37.569860,126.984622


In [102]:
places.loc[12709, 'place.id'] = '117365000'

In [105]:
places = places.drop_duplicates('place.id')

In [106]:
# 네이버 + 트립어드바이저 place
places

,place.id,place.name,place.category,review.count.again,review.allcount,place.count,review.avgcount,is_fch,rating,land.addr,lat,lng
0,11605455,가미분식,종합분식,22.0,18625.0,2121.0,8.781235,0,4.217391,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,37.558661,126.945960
1,10976694,유래회관,"육류,고기요리-일반",18.0,24041.0,3347.0,7.182850,0,4.300000,서울특별시 성동구 홍익동 357 유래회관,37.566438,127.031591
2,11398664,대성갈비,"육류,고기요리-일반",55.0,24041.0,3347.0,7.182850,0,4.216667,서울특별시 성동구 성수동1가 668-21,37.547073,127.044028
3,11663081,철길왕갈비살,"육류,고기요리-일반",34.0,24041.0,3347.0,7.182850,0,4.400000,서울특별시 마포구 창전동 4-2,37.554546,126.931666
4,11677544,황소고집,"육류,고기요리-일반",49.0,24041.0,3347.0,7.182850,0,4.379310,서울특별시 종로구 관철동 11-11,37.568593,126.986046
...,...,...,...,...,...,...,...,...,...,...,...,...
15173,10161356,모모카페 코트야드 서울 남대문,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,서울특별시 중구 남대문로4가 17-23 남대문호텔,37.561200,126.976882
15174,3894901,스타벅스 경복궁역점,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,서울특별시 종로구 통의동 35-41 우진빌딩,37.577200,126.972778
15175,948337,커피愛,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,서울특별시 중구 명동2가 27-1,37.562855,126.985572
15176,9035619,해적선해물부페,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,서울특별시 구로구 고척동 76-160 골든타워빌딩,37.497622,126.864067


In [108]:
places.to_excel(os.path.join("..","..", "data","JS_10_data_places.xlsx"))

# NV + TA Review

In [110]:
nv_review = pd.read_excel(os.path.join("..","..", "data","JS_0502_reviews_seoul_select.xlsx"))
ta_review = pd.read_json(os.path.join("..","..", "data","TA_User_Review_Seoul_all_v2.json"))

In [280]:
places = pd.read_excel(os.path.join("..","..", "data","JS_10_data_places.xlsx"))

In [165]:
ta_review = ta_review.merge(ta_place['location_id'], left_on = 'location.locationId', right_on = 'location_id', how = 'inner')

In [167]:
ta_review.columns

Index(['__typename', 'createdDate', 'helpfulVotes', 'id', 'language',
       'location.__typename', 'location.additionalNames.long',
       'location.additionalNames.longOnlyParent', 'location.locationId',
       'location.name', 'location.parent.additionalNames.long',
       'location.parent.locationId', 'location.parentGeoId',
       'location.placeType', 'photoIds', 'photos', 'rating',
       'socialStatistics.followCount', 'socialStatistics.isFollowing',
       'socialStatistics.isLiked', 'socialStatistics.isReposted',
       'socialStatistics.isSaved', 'socialStatistics.likeCount',
       'socialStatistics.repostCount', 'socialStatistics.tripCount', 'text',
       'title', 'userId', 'userProfile.contributionCounts.sumAllUgc',
       'userProfile.isMe', 'userProfile.isVerified', 'userProfile.userId',
       'location_id'],
      dtype='object')

In [170]:
# 위에서 nv랑 겹치는 ta place 인데스 정보 변경 적용하기
ta_review.loc[ta_review['location.locationId'] == 11736500, 'location.locationId'] = 117365000

In [171]:
# same_place의 location_id에 해당하는 ta_review의 location.locationId를 모두 plcae.id로 바꿔줘야함
same_place

,index,place.id,location_id
0,2,11605455,2241571
1,22,10976694,5926853
2,24,11398664,4032150
3,25,11663081,3922609
4,26,11677544,4033804
...,...,...,...
261,1032,32817375,4784018
262,1035,19723511,9209252
263,1039,33758018,8948116
264,1063,35162551,9029313


In [175]:
for nv_id, ta_id in same_place.values[:,1:]:
    ta_review.loc[ta_review['location.locationId'] == int(ta_id), 'location.locationId'] = nv_id

In [179]:
places.loc[places['place.id'] == '10976694']

,place.id,place.name,place.category,review.count.again,review.allcount,place.count,review.avgcount,is_fch,rating,land.addr,lat,lng
1,10976694,유래회관,"육류,고기요리-일반",18.0,24041.0,3347.0,7.18285,0,4.3,서울특별시 성동구 홍익동 357 유래회관,37.566438,127.031591


In [178]:
ta_review.loc[ta_review['location.locationId'] == '10976694', 'text']

3459    대도식당가 같이 팬을 달구어 소기름으로 준비하고 고기를 굽고 다 먹으면 그 팬에 된...
3460    도선동 성동구에 아주 오래된 음식점으로 대도식당과 근처에서 라이벌인듯 하구요..근데...
3461    동그란 팬을 달구어  소기름으로 기름칠을 하고 고기를 굽습니다.고기를 다 먹으면 그...
3462    다른 고급 한우집 대비 조금은 저렴한 가격에 등심을 먹을 수 있습니다. 직접 구어 ...
3463    비싼 가격만큼 고기의 질은 좋은 음식점입니다.다만 근처에 있는 훨씬 오래된 고기집과...
3464    성동구청과 왕십리에서 가까운 소고기 등심집.\n규모가 커서 대규모 모임에도 적합하다...
3465    상왕십리역에서 내려 성동보건소방향으로 가다 보면 있는 유래회관은 한 건물 전체를 사...
3466    소고기를 전문적으로 판매하는 음식점인데, 숯불이 아닌 두꺼운 후라이팬에 구워먹는 소...
3467    전철역 상왕십리역 2번출구에서 양지사거리에 도착후 성동구보건소방향으로 가다보면 보이...
Name: text, dtype: object

In [182]:
nv_review.loc[nv_review['place.id'] == 10976694, 'body']

279640                                                  NaN
279641                                                  NaN
279642                                                  NaN
279643                                                  NaN
279644                                                  NaN
279645                                       서비스도 좋고 맛도 좋네요
279646                                            한우가 맛있는곳!
279647    좋은점\n1.기다리지않고 먹음\n2. 서버들이 친절한건아니었으나 불친절하지도않았음\...
279648                                       소고기가 꿀맛. 조금 비쌈
279649                                                 none
279650                                                  NaN
279651                                                  NaN
279652                                                  NaN
279653                                                    굿
279654                                   가성비 나쁨 비싸지만 고기는 쏘쏘
279655                                                  NaN
279656                                  

In [183]:
# nv, ta columns 선택하기

In [184]:
nv_review.columns

Index(['rating', 'body', 'visitCount', 'viewCount', 'thumbnail', 'status',
       'media', 'id', '__typename', 'idno', 'visitedDate.date',
       'visitedDate.displayDate', 'visitedDate.displayDateTime',
       'visitedDate.__typename', 'place.id', 'place.name', 'place.category',
       'place.categoryCode', 'place.categoryCodes', 'place.phone',
       'place.address', 'place.__typename', 'isseoul', 'media_count'],
      dtype='object')

In [185]:
ta_review.columns

Index(['__typename', 'createdDate', 'helpfulVotes', 'id', 'language',
       'location.__typename', 'location.additionalNames.long',
       'location.additionalNames.longOnlyParent', 'location.locationId',
       'location.name', 'location.parent.additionalNames.long',
       'location.parent.locationId', 'location.parentGeoId',
       'location.placeType', 'photoIds', 'photos', 'rating',
       'socialStatistics.followCount', 'socialStatistics.isFollowing',
       'socialStatistics.isLiked', 'socialStatistics.isReposted',
       'socialStatistics.isSaved', 'socialStatistics.likeCount',
       'socialStatistics.repostCount', 'socialStatistics.tripCount', 'text',
       'title', 'userId', 'userProfile.contributionCounts.sumAllUgc',
       'userProfile.isMe', 'userProfile.isVerified', 'userProfile.userId',
       'location_id'],
      dtype='object')

In [287]:
nv = nv_review[['idno', 'rating', 'place.id']]

In [288]:
ta= ta_review[['userId', 'rating', 'location.locationId']]

In [289]:
ta.columns = ['idno', 'rating', 'place.id']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415515 entries, 0 to 81630
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   idno      415515 non-null  object 
 1   rating    415515 non-null  float64
 2   place.id  415515 non-null  object 
dtypes: float64(1), object(2)
memory usage: 12.7+ MB


In [290]:
rating = pd.concat([nv, ta], axis = 0)

In [481]:
rating.to_excel(os.path.join("..","..", "data","JS_10_data_rating.xlsx"), index = False)

In [219]:
rating_matrix['rating'] = rating_matrix['rating'].astype(float)

In [220]:
rating_matrix = pd.pivot_table(rating_matrix, index = 'idno', columns = 'place.id', values = 'rating', fill_value=0)

In [225]:
rating_matrix.to_pickle(os.path.join("..","..", "data","JS_10_data_rating_matrix.pickle"))

In [474]:
rating_matrix.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65804 entries, 0002C7C9BF0686DB770487FD63B5C6F5 to zzi2
Columns: 15439 entries, 776544 to 880014619
dtypes: float64(7856), int64(7583)
memory usage: 7.6+ GB


In [ ]:
rating_matrix.to_excel(os.path.join("..","..", "data","JS_10_data_rating_matrix.xlsx"), index = False)

# places NaN 채우기

In [281]:
places

,place.id,place.name,place.category,review.count.again,review.allcount,place.count,review.avgcount,is_fch,rating,land.addr,lat,lng
0,11605455,가미분식,종합분식,22.0,18625.0,2121.0,8.781235,0,4.217391,서울특별시 서대문구 대현동 54-1 무궁화상가아파트,37.558661,126.945960
1,10976694,유래회관,"육류,고기요리-일반",18.0,24041.0,3347.0,7.182850,0,4.300000,서울특별시 성동구 홍익동 357 유래회관,37.566438,127.031591
2,11398664,대성갈비,"육류,고기요리-일반",55.0,24041.0,3347.0,7.182850,0,4.216667,서울특별시 성동구 성수동1가 668-21,37.547073,127.044028
3,11663081,철길왕갈비살,"육류,고기요리-일반",34.0,24041.0,3347.0,7.182850,0,4.400000,서울특별시 마포구 창전동 4-2,37.554546,126.931666
4,11677544,황소고집,"육류,고기요리-일반",49.0,24041.0,3347.0,7.182850,0,4.379310,서울특별시 종로구 관철동 11-11,37.568593,126.986046
...,...,...,...,...,...,...,...,...,...,...,...,...
15172,10161356,모모카페 코트야드 서울 남대문,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,서울특별시 중구 남대문로4가 17-23 남대문호텔,37.561200,126.976882
15173,3894901,스타벅스 경복궁역점,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,서울특별시 종로구 통의동 35-41 우진빌딩,37.577200,126.972778
15174,948337,커피愛,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,서울특별시 중구 명동2가 27-1,37.562855,126.985572
15175,9035619,해적선해물부페,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,서울특별시 구로구 고척동 76-160 골든타워빌딩,37.497622,126.864067


In [332]:
rating['place.id'] = rating['place.id'].astype(str)

In [333]:
temp = rating.groupby('place.id')['idno'].count().reset_index()

In [337]:
temp.columns = ['place.id', 'review.count']

In [349]:
places = places.merge(temp, on = 'place.id', how = 'left')

In [350]:
places.loc[places['review.count'].isnull()]

,place.id,place.name,place.category,review.count.again,review.allcount,place.count,review.avgcount,is_fch,rating,land.addr,lat,lng,review.count
6745,none,역사가 우리를 망쳐 놨지만 그래도 상관없다,한정식-일반,NaN,3215.0,430.0,7.476744,0,4.146667,서울특별시 용산구 용산동2가 17,37.545679,126.985451,NaN


In [352]:
places = places.drop(6745)

In [356]:
places = places[['place.id', 'place.name', 'place.category', 'rating','review.count', 'is_fch', 'lat', 'lng', 'land.addr']]

In [358]:
places.to_excel(os.path.join('..', '..', 'data', 'JS_10_data_places_v2.xlsx'))

# Review

In [367]:
nv = nv_review[['idno', 'rating', 'place.id', 'id']]
ta= ta_review[['userId', 'rating', 'location.locationId', 'id']]

In [370]:
nv = nv.rename(columns = {'id' : 'review.id'})
ta.columns = ['idno', 'rating', 'place.id', 'review.id']

In [372]:
reviews = pd.concat([nv, ta], axis = 0)

In [375]:
reviews.head()

,idno,rating,place.id,review.id
0,4TXey,4.5,1150710065,5ec7ec658160f100109a8368
1,awWC,5.0,1150710065,5d14eaa963951f37e4243fa5
2,1Unzg,5.0,1150710065,5e69024bf896230011648a61
3,LRVl,5.0,1150710065,5e4f98b95543c700113e18e7
4,ESN1,5.0,1150710065,5c8528bdf9e6b96d632ae302


In [373]:
nv_review.loc[0]

rating                                                          4.5
body                                                            NaN
visitCount                                                        2
viewCount                                                         1
thumbnail                                                      none
status                                                    published
media                                                            []
id                                         5ec7ec658160f100109a8368
__typename                                                   Review
idno                                                          4TXey
visitedDate.date                              2020. 05. 21 20:42:00
visitedDate.displayDate                                     05.21 목
visitedDate.displayDateTime                    2020. 05. 21 목 20:42
visitedDate.__typename                                  VisitedTime
place.id                                        

In [417]:
nv_review['photonum'] = nv_review['media'].apply(lambda x: x.count('IMAGE'))

In [424]:
nv_review['visitedDate.date'].sort_values()

155333    1118. 12. 29 11:58:00
133583    1410. 04. 20 19:18:00
306682    1991. 02. 01 00:00:00
106279    1999. 07. 27 00:00:00
324673    2000. 01. 13 13:26:00
                  ...          
151865    2020. 05. 30 23:49:43
32948     2020. 05. 31 00:22:00
330452    2020. 05. 31 00:29:49
310458    2020. 05. 31 00:31:16
243397    2020. 05. 31 00:47:19
Name: visitedDate.date, Length: 333884, dtype: object

In [433]:
nv_review.loc[155333, 'visitedDate.date'] = '2018-12-29 11:58:00'
nv_review.loc[133583, 'visitedDate.date'] = '2010-04-20 19:18:00'

In [435]:
nv_review['createdDate'] = nv_review['visitedDate.date'].astype(str).astype('datetime64') 

In [452]:
ta_review['photonum'] = ta_review['photos'].apply(lambda x: len(x))

In [438]:
ta_review['createdDate'] = ta_review['createdDate'].astype(str).astype('datetime64')

In [363]:
ta_review.loc[0]

__typename                                                                             Review
createdDate                                                                        2018-03-14
helpfulVotes                                                                                0
id                                                                                  566403778
language                                                                                   ko
location.__typename                                                       LocationInformation
location.additionalNames.long                                         Marco Polo, South Korea
location.additionalNames.longOnlyParent                                           South Korea
location.locationId                                                                    788542
location.name                                                                      Marco Polo
location.parent.additionalNames.long                        

In [454]:
nv_review = nv_review[['id', 'rating', 'body', 'idno', 'place.id', 'place.name', 'photonum', 'createdDate']]

In [455]:
ta_review = ta_review[['id', 'rating', 'text', 'userId', 'location.locationId', 'location.name', 'photonum', 'createdDate']]

In [457]:
ta_review.columns = ['id', 'rating', 'body', 'idno', 'place.id', 'place.name', 'photonum', 'createdDate']

In [463]:
reviews = pd.concat([nv_review, ta_review], axis = 0)

In [466]:
reviews['place.id'] = reviews['place.id'].astype(str)

In [468]:
reviews.to_excel(os.path.join('..', '..', 'data', 'JS_10_data_review.xlsx'), index = False)